### **Visualização do dataset**

In [1]:
import sys
sys.path.append('/home/guilherme/AIEnvironment/SentimentAnalysis/src/data_preparation')
sys.path.append('/home/guilherme/AIEnvironment/SentimentAnalysis/src/feature_engineering')
from data_exploration import load_data, read_text


#### **Carregando a base de dados e visualizando a primeira avaliação**

In [2]:
df = load_data("/home/guilherme/AIEnvironment/SentimentAnalysis/data/processed/df_imdb.csv")
text = df['Reviews'][0]
first_text = read_text(text)
for text in first_text:
    print(text)

Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from t
he terrific sea rescue sequences, of which there are very few I just did not care about an
y of the characters. Most of us have ghosts in the closet, and Costners character are real
ized early on, and then forgotten until much later, by which time I did not care. The char
acter we should really care about is a very cocky, overconfident Ashton Kutcher. The probl
em is he comes off as kid who thinks hes better than anyone else around him and shows no s
igns of a cluttered closet. His only obstacle appears to be winning over Costner. Finally 
when we are well past the half way point of this stinker, Costner tells us all about Kutch
ers ghosts. We are told why Kutcher is driven to be the best with no prior inkling or fore
shadowing. No magic here, it was all I could do to keep from turning it off an hour in.


#### Resposta: 
- Como pode-se perceber, são apenas textos, a avaliação inclusive mostra uma crítica negativa. Para que algoritmos de Machine Learning apropriados possam ser aplicados, é preciso tratar os textos de forma numérica.

### **Pré-processamento: Aplicando técnicas de NLP (Natural Language Processing)**

#### **1. Bag of Words**

- Abordagem que consiste em realizar uma contagem de palavras, ou seja, transformar o texto em um vetor de números. Processo de extração de características de texto para que uma forma estruturada possa ser utilizada para algoritmos de Machine Learning.

Obs: A técnica Bag of Words não leva em consideração a ordem das palavras, apenas a frequência com que aparecem, não sendo apropriada para textos longos, também não realiza nenhum tratamento nos textos. 
